# Лабораторная работа 4. Часть 1. Ранжирование.



Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 9 баллов. Сдавать задание после указанного в lk срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).



## Знакомство с данными

### Ранжирование организаций по пользовательскому запросу

Что мы обычно делаем, когда нам нужно найти определённое место, но не знаем его местоположения? Используем поиск на картах.

В этой лабораторной работе вам будет необходимо построить небольшую поисковую систему, позволяющую отранжировать организации по запросу пользователя.

Для обучения вам даны 2000 запросов и более 13 тысяч найденных по ним организаций. Для каждой пары "запрос — организация" была посчитана релевантность, по которой и происходит ранжирование.

**(2 балла) Задание 1.** Загрузите [данные](https://disk.yandex.ru/d/Bf3P4H8FDYe7-g) о запросах и их релевантности (*train.csv*), а также информацию об организациях (*train_org_information.json*) и рубриках (*train_rubric_information.json*)

Для дальнейших экспериментов необходимо посчитать небольшой набор базовых факторов. С использованием информации о запросах и организациях, посчитайте факторы, которые на ваш взгляд будут полезными для предсказания релевантности.

Примерами текстовых факторов могут служить:
 - кол-во слов в запросе и названии организации;
 - пословные/N-граммные пересечения слов запроса и названия организации (также можно использовать синонимы названия организации и адрес организации): кол-во слов в пересечении, [мера Жаккара](https://en.wikipedia.org/wiki/Jaccard_index) и пр.;
 - кол-во различных синонимичных названий организации (поле *names* в описании организации);
 - One-hot-encoded язык запроса.
 
По информации о географическом положении:
 - факт совпадения региона, где задавался запрос и региона организации;
 - координаты показанной области;
 - размеры показанной области;
 - меры, характеризующие близость координат организации к показанному окну: расстояние до центра области и другие.
 
Факторы, описывающие организацию:
 - one-hot-encoding фактор cтраны или региона организации (важно: не используйте one-hot-encoding факторы, в которых больше 10 значений; если в факторе слишком много значений, ограничьтесь, например, только самыми популярными категориями)
 - кол-во рабочих дней в неделе и общая продолжительность работы (поле *work_intervals* в описании организации)
 - кол-во рубрик (поле *rubrics* в описании организации)
 
![](https://miro.medium.com/max/1500/0*FwubnnoNlt6Coo9j.png)

В этом задании не нужно использовать многомерные представления текстовой информации (tfidf и прочие embeddings) и информацию о кликах (*train_clicks_information.json*). Придумывать сверхсложные факторы тоже необязательно.

Вы можете реализовать описанные выше факторы и/или придумать свои. Но зачастую такие простые признаки могут приносить наибольшую пользу.

В итоге у вас должно получиться от 15 до 50 факторов, характеризующих запрос и организацию и покрывающих основные источники данных (кроме кликов). Это наш основной датасет, который будет использоваться в экспериментах.



In [1]:
#!pip3 install wldhx.yadisk-direct

In [2]:
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/Bf3P4H8FDYe7-g) -o ranking_data.zip

In [3]:
# !unzip ranking_data.zip

In [4]:
import json
import langdetect
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
import logging
import random
from tqdm.auto import tqdm
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
np.random.seed(2022)
random.seed(2022)

In [5]:
train_data = pd.read_csv('./2021_sem3_lab_04_part_01_ranking_data/train.csv')
train_data.sample(5)

,query_id,query,region,org_name,org_id,window_center,window_size,relevance
21872,29498,крит Marilena hotel 4* расположение,18,"На час, Аманда отель на Адмиралтейской",1749152458,"34.359688,61.789036","0.466091,0.149715",0.00
25881,33228,сад 103 александра невского 7,2,Детский сад № 38 Румяные щечки,20242747624,"38.924181,55.987513","0.040855,0.012437",0.00
22255,29787,листок горно-алтайск,11319,"Россельхозбанк, банкомат",1027308246,"85.951597,51.958287","0.026178,0.008617",0.00
28004,34964,"ул. Талсинская, д.23 (ТЦ ""Сиреневый"")",213,"Ситилинк мини, пункт выдачи",1292610377,"37.617671,55.755768","1.620483,0.451494",0.00
25714,33145,русклимат,47,Русклимат,45671709168,"43.845260,56.316998","0.252857,0.076702",0.02


In [6]:
train_data.shape

(29274, 8)

In [7]:
for i in np.random.randint(10, 100, 3):
    for col, data in zip(train_data.columns, train_data.iloc[i].values):
        print(col, ":", data)
    print()

query_id : 76
query : кладбище
region : 28313
org_name : Кладовище № 2
org_id : 1019383718
window_center : 36.112721,49.953447
window_size : 0.023285,0.008106
relevance : 0.0

query_id : 73
query : Гостиницы
region : 24897
org_name : Готель Хеллі
org_id : 1076500882
window_center : 23.506196,49.275280
window_size : 0.015378,0.012322
relevance : 0.14

query_id : 73
query : Гостиницы
region : 24897
org_name : Готель Весна
org_id : 1188713685
window_center : 23.506196,49.275280
window_size : 0.015378,0.012322
relevance : 0.14



In [8]:
i

51

In [9]:
for _, row in train_data[train_data.query_id == train_data.iloc[i].query_id].iterrows():
    print("query:", row.query)
    print("org_name", row.org_name)
    print("relevance", row.relevance)
    print()

query: Гостиницы
org_name Готель Парк Трускавець
relevance 0.14

query: Гостиницы
org_name Готель Mirotel Resort & Spa
relevance 0.14

query: Гостиницы
org_name Курортно-готельний комплекс Свитязь
relevance 0.14

query: Гостиницы
org_name Готель Весна
relevance 0.14

query: Гостиницы
org_name Клуб pH
relevance 0.14

query: Гостиницы
org_name Відпочинковий комплекс Клейнод
relevance 0.14

query: Гостиницы
org_name Старий Відень
relevance 0.14

query: Гостиницы
org_name Готель у Олега
relevance 0.14

query: Гостиницы
org_name Готель Вiан
relevance 0.14

query: Гостиницы
org_name Magnet hotel
relevance 0.14

query: Гостиницы
org_name Ресторан Оріон
relevance 0.14

query: Гостиницы
org_name Готель Club Ph
relevance 0.14

query: Гостиницы
org_name Міні-готель Джем
relevance 0.14

query: Гостиницы
org_name Готель Маріот Медікал центр
relevance 0.14

query: Гостиницы
org_name Готель Мальви
relevance 0.14

query: Гостиницы
org_name Санаторій Весна
relevance 0.14

query: Гостиницы
org_name Готе

In [10]:
org_data = pd.read_json('./2021_sem3_lab_04_part_01_ranking_data/train_org_information/train_org_information.json', 
                        orient='index', 
                        convert_axes=False, 
                        convert_dates=False)


rubric_data = pd.read_json('./2021_sem3_lab_04_part_01_ranking_data/train_rubric_information/train_rubric_information.json', 
                           orient='index', 
                           convert_axes=False, 
                           convert_dates=False)

In [11]:
org_data.head()

,names,rubrics,work_intervals,address
1255014404,"[{'value': {'locale': 'en', 'value': 'Bely Bok...","[20277, 20679, 21237]","[{'time_minutes_begin': 540, 'day': 'everyday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
1111883782,"[{'value': {'locale': 'en', 'value': 'Vyborgsk...",[30723],"[{'time_minutes_begin': 510, 'day': 'saturday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
39713767434,"[{'value': {'locale': 'en', 'value': 'Sberbank...",[30348],"[{'time_minutes_begin': 570, 'day': 'saturday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
1336016908,"[{'value': {'locale': 'en', 'value': 'Filial b...","[30057, 30111, 30114]","[{'time_minutes_begin': 480, 'day': 'saturday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
149331116064,"[{'value': {'locale': 'ru', 'value': 'Школа'}}]",[30723],[],"{'region_code': 'RU', 'formatted': {'locale': ..."


In [12]:
rubric_data.head()

,keywords,phrases,descriptions,names
273147906,"[{'locale': 'ru', 'value': 'велосипед парковка...","[{'locale': 'de', 'value': 'Parkplatz für Fahr...","[{'value': {'locale': 'tr', 'value': 'Bisiklet...","[{'locale': 'en', 'value': 'Bicycle stand'}, {..."
30723,"[{'locale': 'tr', 'value': 'liseler, ortaokul,...","[{'locale': 'de', 'value': 'Bildungseinrichtun...","[{'value': {'locale': 'tr', 'value': 'Çocuklar...","[{'locale': 'de', 'value': 'Allgemeinbildende ..."
30724,"[{'locale': 'tr', 'value': 'arastırma, bilim, ...","[{'locale': 'tr', 'value': 'Bilim Arastırma En...","[{'value': {'locale': 'tr', 'value': 'Bilim ar...","[{'locale': 'tr', 'value': 'Bilim Araştırma En..."
30725,"[{'locale': 'tr', 'value': 'eğitim kontrol'}]","[{'locale': 'en', 'value': 'Board of Education...","[{'value': {'locale': 'tr', 'value': 'Eğitim k...","[{'locale': 'it', 'value': 'Amministrazione de..."
30731,"[{'locale': 'ru', 'value': 'Новые технологии -...","[{'locale': 'en', 'value': 'implementation of ...","[{'value': {'locale': 'tr', 'value': 'Nanotekn...","[{'locale': 'de', 'value': 'Innovative Technol..."


In [13]:
with open('./2021_sem3_lab_04_part_01_ranking_data/train_org_information/train_org_information.json', 'r') as read_file:
    train_org_json = json.load(read_file)

In [14]:
with open('./2021_sem3_lab_04_part_01_ranking_data/train_rubric_information/train_rubric_information.json') as read_file:
    train_rubric_json = json.load(read_file)

In [111]:
def get_words_from_phrase(x) -> str:
    return re.sub(r"[^A-Za-zА-Яа-я]+", ' ', x).lower().split()

def get_intersection_words_count(x) -> int:
    return len(set(get_words_from_phrase(x[0])) & set(get_words_from_phrase(x[1])))

def get_synonymous_names_count(x) -> int:
    node = train_org_json.get(str(x))

    return len(node["names"]) if node is not None else 0

def get_encoded_langs():
    langs = train_data['query'].apply(lambda x: langdetect.detect(x))
    sortedLangs = [lang for lang, _ in Counter(langs).most_common()]
    langsEncoder = OneHotEncoder(sparse=False).fit(langs.values.reshape(-1,1))
    encodedLangs = langsEncoder.transform(langs.values.reshape(-1,1))
    langsIdx = [np.where(langsEncoder.categories_[0] == lang)[0][0] for lang in sortedLangs[:10]]
    
    newEncodedLangs = np.zeros((encodedLangs.shape[0], 11))
    for i, idx in enumerate(langsIdx):
        newEncodedLangs[:, i] = encodedLangs[:, idx]
    for row in newEncodedLangs:
        if row.sum() == 0:
            row[-1] = 1
    return newEncodedLangs

def get_rubrics_count(x) -> int:
    node = train_org_json.get(str(x))
    return len(node["rubrics"]) if node is not None else 0

def get_work_time(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        work_interval_sum = 0
        for work_interval in node["work_intervals"]:
            work_interval_sum += work_interval["time_minutes_end"] - work_interval["time_minutes_begin"]
        if len(node["work_intervals"]) != 0:
            return work_interval_sum / len(node["work_intervals"])
    return 0

def get_days_count(x):
    node = train_org_json.get(str(x))
    if node is not None:
        daysCount = 0
        for work_interval in node["work_intervals"]:
            day = work_interval["day"]
            if day == "everyday":
                daysCount = max(daysCount, 7)
            elif day == "weekdays":
                daysCount = max(daysCount, 5)
            elif day == "weekend":
                daysCount = max(daysCount, 2)
            elif day in ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]:
                daysCount = 1
            else:
                daysCount = 0
        return daysCount
    return 0

def get_region_code(x) -> str:
    node = train_org_json.get(str(x))

    return node["address"]["region_code"] if node is not None else 0

def get_encoded_region_code():
    region_codes = train_data["org_id"].apply(get_region_code).values
    sorted_region_codes = [code for code, _ in Counter(region_codes).most_common()[:10]]
    codesEncoder = OneHotEncoder(sparse=False).fit(region_codes.reshape(-1,1))
    encodedCodes = codesEncoder.transform(region_codes.reshape(-1,1))
    codesIdx = [np.where(codesEncoder.categories_[0] == code)[0][0] for code in sorted_region_codes[:10]]
    
    new_encoded_codes = np.zeros((encodedCodes.shape[0], 11))
    for i, idx in enumerate(codesIdx):
        new_encoded_codes[:, i] = encodedCodes[:, idx]
        
    for row in new_encoded_codes:
        if row.sum() == 0:
            row[-1] = 1
            
    return new_encoded_codes

In [112]:
df = pd.DataFrame()

df['query_id'] = train_data['query_id']
df['query_words_count'] = train_data['query'].apply(lambda query: len(get_words_from_phrase(query)))
df['org_name_words_count'] = train_data['org_name'].apply(lambda name: len(get_words_from_phrase(name)))
df['intersection_words_count'] = train_data[['query', 'org_name']].apply(get_intersection_words_count, axis=1)
df['jaccard_score'] = df['intersection_words_count'] / (df['query_words_count'] + df['org_name_words_count'] - df['intersection_words_count'])
df['synonymous_names_count'] = train_data['org_id'].apply(get_synonymous_names_count)
df['rubrics_count'] = train_data['org_id'].apply(get_rubrics_count)
df['worktime'] = train_data['org_id'].apply(get_work_time)
df['workdays_count'] = train_data['org_id'].apply(get_days_count)
df["relevance"] = train_data["relevance"]

In [113]:
df.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count,relevance
0,11,7,7,1,0.076923,5,1,240.0,1,0.0
1,11,7,7,1,0.076923,5,1,540.0,5,0.0
2,11,7,4,1,0.100000,7,1,480.0,5,0.0
3,11,7,4,1,0.100000,7,1,240.0,1,0.0
4,11,7,2,1,0.125000,3,1,540.0,5,0.0


## Ранжирование

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.
3. **Listwise подход**. Этот подход использует все объекты для вычисления функции потерь, стараясь явно оптимизировать правильный порядок.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

gssВ рамках нашей задачи «документом» будет являться организация.

Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание, что разбивать необходимо множество запросов, а не строчки датасета.

In [18]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(test_size = .3, n_splits = 1).split(df, groups=df.query_id)
X_train_inds, X_test_inds = next(gss)

new_train_data = df.iloc[X_train_inds]
X_train = new_train_data.loc[:, ~new_train_data.columns.isin(['relevance'])]
y_train = new_train_data.loc[:, new_train_data.columns.isin(['query_id', 'relevance'])]

groups = train_data.groupby('query_id').size().to_frame('size')['size']

new_test_data = df.iloc[X_test_inds]

X_test = new_test_data.loc[:, ~new_test_data.columns.isin(['relevance'])]
y_test = new_test_data.loc[:, new_test_data.columns.isin(['query_id', 'relevance'])]

In [19]:
X_train_inds

array([    0,     1,     2, ..., 29257, 29258, 29259])

Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам, где разбиение должно быть по запросам, а не строчкам датасета.

###  Ранжируем с XGBoost и CatBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

Реализованные в CatBoost ранжирующие функции потерь можной найти [здесь](https://catboost.ai/docs/concepts/loss-functions-ranking.html#groupwise-metrics).

In [20]:
def get_fold_indexes(train_data, n_folds=5):
    val_gss = GroupShuffleSplit(n_splits=n_folds, test_size=0.2, random_state=457)

    fold_indexes = []
    for train_idx, test_idx in val_gss.split(train_data, groups=train_data['query_id']):
        fold_indexes.append({'train_idx': train_idx, 'test_idx': test_idx})
        
    return fold_indexes

def get_specify_fold(X, fold_indexes, fold_number):
    X_val_train = X.iloc[fold_indexes[fold_number]['train_idx']].loc[:, ~X.columns.isin(['relevance'])]
    y_val_train = X.iloc[fold_indexes[fold_number]['train_idx']].loc[:, X.columns.isin(['query_id','relevance'])]
    
    X_val_test = X.iloc[fold_indexes[fold_number]['test_idx']].loc[:, ~X.columns.isin(['relevance'])]
    y_val_test = X.iloc[fold_indexes[fold_number]['test_idx']].loc[:, X.columns.isin(['query_id','relevance'])]
    return (X_val_train, y_val_train), (X_val_test, y_val_test)

In [114]:
def DCG(y_true, y_pred, k = 10):
    
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2) + 1e-6
    return np.sum(gains / discounts)

def nDCG(y_true, y_pred, k=10):
    
    best = DCG(y_true, y_true, k) + 1e-6
    actual = DCG(y_true, y_pred, k)
    return actual / best

def nDCG_query(y_true_group, y_pred_group):
    return np.mean([nDCG(y_true_group[query_id], y_pred_group[query_id]) for query_id in y_pred_group.index.values])


In [115]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['query_id'])])

In [116]:
import catboost

def cross_val_scores(model, new_train_data, n_folds=5, is_grouping=False, is_catboost=False):
    ndcg_score = []
    fold_train_indexes = get_fold_indexes(new_train_data)
    for i in range(n_folds):
        (X_val_train, y_val_train), (X_val_test, y_val_test) = get_specify_fold(new_train_data, fold_train_indexes, i)
        
        fit_parameters = {
           'X': X_val_train.loc[:, ~X_val_train.columns.isin(['query_id'])], 
            'y': y_val_train.relevance,
            'verbose': True
        }
    
        if is_grouping:
            fit_parameters['group'] = X_val_train.groupby('query_id').size().to_frame('size')['size'].to_numpy()
            
        if is_catboost:
            fit_parameters['data'] = fit_parameters.pop('X')
            fit_parameters['label'] = fit_parameters.pop('y')
            fit_parameters['group_id'] = X_val_train['query_id']
            del fit_parameters['verbose']
        
            train = catboost.Pool(**fit_parameters)
            model.fit(train, silent=True)
        else:
            model.fit(**fit_parameters)

        predictions = (X_val_test.groupby('query_id').apply(lambda x: predict(model, x)))
        ndcg_score.append(nDCG_query(y_val_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions))
    return ndcg_score

In [117]:
fold_train_indexes = get_fold_indexes(new_train_data)
(X_val_0, y_val_0), (X_val_1, y_val_1) = get_specify_fold(new_train_data, fold_train_indexes, 0)



**(3 балла) Задание 2.** Попробуйте различные функции потерь (регрессионные и ранжирующие) для моделей XGBoost и CatBoost. Настройте основные параметры моделей (глубина, кол-во деревьев, глубина, скорость обучения, регуляризация).  
Сравните построенные модели с точки зрения метрики nDCG на контроле и проанализируйте полученные результаты:
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?

In [119]:
import itertools
from tqdm.auto import tqdm
import random
import copy

def grid_search(model, hyper_parameters_grid, cross_val_scores_func):
    results = pd.DataFrame()
    full_parameters_grid = list(itertools.product(*hyper_parameters_grid.values()))
    
    print(f'full gridsearch len is {len(full_parameters_grid)}')
    
    if len(full_parameters_grid) > 100:
        full_parameters_grid = random.sample(full_parameters_grid, 100)
       
    # тут надобы было распараллелеить ..
    for curr_params in tqdm(full_parameters_grid, total=len(full_parameters_grid)):
        current_parametrs = {list(hyper_parameters_grid.keys())[i]: param for i, param in enumerate(curr_params)}
        
        cur_model = copy.deepcopy(model)
        cur_model.set_params(**current_parametrs)
        current_metric = np.mean(cross_val_scores_func(cur_model, new_train_data))
        current_parametrs['nDCDG_cv'] = current_metric

        results = results.append(current_parametrs, ignore_index = True)    
    
    results.sort_values(
        by="nDCDG_cv",
        inplace=True,
        ascending=False
    )
     
    return results
    

In [120]:
import xgboost as xgb

model = xgb.XGBRegressor(
                objective='reg:squarederror',
                verbosity=0
            )


df = grid_search(model, {'max_depth' : range(1,4), 'learning_rate': [1e-3, 1e-2]}, cross_val_scores)

full gridsearch len is 6


  0%|          | 0/6 [00:00<?, ?it/s]

In [121]:
df

,learning_rate,max_depth,nDCDG_cv
0,0.001,1.0,0.906752
1,0.010,1.0,0.906752
2,0.001,2.0,0.897273
4,0.001,3.0,0.852851
3,0.010,2.0,0.846812
5,0.010,3.0,0.794008


## xgboost

In [ ]:
import xgboost as xgb

### reg:squarederror

In [ ]:
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])], y_train.relevance, verbose=0)

In [ ]:
predictions = (X_test.groupby('query_id').apply(lambda x: predict(model, x)))

In [33]:
nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)

0.6670563883058807

In [101]:
params_grid = {
    'max_depths': range(1, 2, 9),
    'learning_rate': [1e-3, 1e-2, 1e-1, 1],
    'n_estimators': [10, 25, 50, 75, 100],
    'reg_alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1],
    'reg_lambda': [1e-4, 1e-3, 1e-2, 1e-1, 1]
}


model = xgb.XGBRegressor(
            objective='reg:squarederror',
            verbosity=0
)

grid_search(model, params_grid, cross_val_scores).head(10)

full gridsearch len is 500


  0%|          | 0/100 [00:00<?, ?it/s]

,learning_rate,max_depths,nDCDG_cv,n_estimators,reg_alpha,reg_lambda
29,0.001,1.0,0.892627,50.0,1.0000,1.0000
46,0.001,1.0,0.886678,10.0,1.0000,0.0100
23,0.001,1.0,0.886678,10.0,1.0000,0.0001
32,0.001,1.0,0.885873,25.0,1.0000,0.0001
44,0.010,1.0,0.834622,50.0,1.0000,0.1000
81,0.010,1.0,0.828947,50.0,1.0000,0.0010
15,0.010,1.0,0.828702,50.0,1.0000,0.0100
52,0.010,1.0,0.825110,75.0,1.0000,0.1000
0,0.100,1.0,0.817744,10.0,1.0000,0.0100
61,0.001,1.0,0.816676,10.0,0.0001,1.0000


### rank:pairwise



In [102]:
import functools

model = xgb.XGBRanker(
            objective='rank:pairwise',
        )

grid_search(model, params_grid, functools.partial(cross_val_scores, is_grouping=True)).head(10)

full gridsearch len is 500


  0%|          | 0/100 [00:00<?, ?it/s]

,learning_rate,max_depths,nDCDG_cv,n_estimators,reg_alpha,reg_lambda
47,0.010,1.0,0.727773,10.0,1.0000,0.0010
32,0.010,1.0,0.727720,10.0,0.0001,0.0010
56,0.010,1.0,0.727153,10.0,1.0000,0.0100
86,0.100,1.0,0.726716,10.0,1.0000,0.0001
22,0.100,1.0,0.726224,10.0,0.0001,0.0001
54,0.100,1.0,0.726089,10.0,1.0000,0.0100
98,0.010,1.0,0.726001,10.0,1.0000,0.1000
77,0.001,1.0,0.725533,10.0,0.0100,0.0001
0,0.100,1.0,0.725499,10.0,0.0010,0.1000
44,0.010,1.0,0.725290,25.0,1.0000,1.0000


### rank:map

In [105]:
model = xgb.XGBRanker(
            objective='rank:map',
        )
grid_search(model, params_grid, functools.partial(cross_val_scores, is_grouping=True)).head(10)

full gridsearch len is 500


  0%|          | 0/100 [00:00<?, ?it/s]

,learning_rate,max_depths,nDCDG_cv,n_estimators,reg_alpha,reg_lambda
66,0.010,1.0,0.730316,10.0,0.0100,0.1000
38,0.010,1.0,0.729460,10.0,0.0001,0.1000
42,0.010,1.0,0.729449,10.0,0.0010,0.1000
49,0.010,1.0,0.728119,10.0,0.0100,0.0010
17,0.010,1.0,0.727643,10.0,0.1000,0.0100
90,0.001,1.0,0.727601,10.0,0.0010,0.0001
37,0.001,1.0,0.727577,10.0,0.1000,0.0100
84,0.010,1.0,0.726698,10.0,1.0000,0.0100
64,0.001,1.0,0.726325,25.0,1.0000,1.0000
7,0.010,1.0,0.725587,10.0,1.0000,0.0001


### rank:ndcg

In [106]:
model = xgb.XGBRanker(
            objective='rank:ndcg',
        )
grid_search(model, params_grid, functools.partial(cross_val_scores, is_grouping=True)).head(10)

full gridsearch len is 500


  0%|          | 0/100 [00:00<?, ?it/s]

,learning_rate,max_depths,nDCDG_cv,n_estimators,reg_alpha,reg_lambda
0,1.000,1.0,0.906752,75.0,0.0001,0.0001
63,1.000,1.0,0.906752,100.0,1.0000,0.0010
73,0.001,1.0,0.906752,25.0,0.0010,0.0001
72,0.001,1.0,0.906752,25.0,0.0001,0.0001
71,0.010,1.0,0.906752,75.0,0.0100,0.0100
70,1.000,1.0,0.906752,75.0,1.0000,0.0010
69,1.000,1.0,0.906752,100.0,0.0010,0.0100
68,0.001,1.0,0.906752,100.0,0.0010,1.0000
67,0.010,1.0,0.906752,75.0,1.0000,1.0000
66,0.010,1.0,0.906752,10.0,1.0000,0.1000


## Catboost

In [107]:
import catboost

In [108]:
params_grid_cat = {
    'depth': range(1, 2, 9),
    'learning_rate': [1e-3, 1e-2, 1e-1, 1],
    'n_estimators': [10, 25, 50, 75, 100],
    'reg_lambda': [1e-4, 1e-3, 1e-2, 1e-1, 1]
}


model = catboost.CatBoostRanker(loss_function='YetiRank')

grid_search(model, params_grid_cat, functools.partial(cross_val_scores, is_catboost=True)).head(15)

full gridsearch len is 100


  0%|          | 0/100 [00:00<?, ?it/s]

,depth,learning_rate,nDCDG_cv,n_estimators,reg_lambda
0,1.0,0.001,0.863246,10.0,0.0001
2,1.0,0.001,0.863246,10.0,0.0100
3,1.0,0.001,0.863246,10.0,0.1000
4,1.0,0.001,0.863246,10.0,1.0000
1,1.0,0.001,0.863246,10.0,0.0010
29,1.0,0.010,0.852066,10.0,1.0000
5,1.0,0.001,0.843890,25.0,0.0001
6,1.0,0.001,0.843890,25.0,0.0010
7,1.0,0.001,0.843890,25.0,0.0100
8,1.0,0.001,0.843890,25.0,0.1000


In [110]:
params_grid_cat = {
    'depth': range(1, 2, 9),
    'learning_rate': [1e-3, 1e-2, 1e-1, 1],
    'n_estimators': [10, 25, 50, 75, 100],
    'reg_lambda': [1e-4, 1e-3, 1e-2, 1e-1, 1]
}


model = catboost.CatBoostRanker(loss_function='YetiRankPairwise')

grid_search(model, params_grid_cat, functools.partial(cross_val_scores, is_catboost=True)).head(15)

full gridsearch len is 100


  0%|          | 0/100 [00:00<?, ?it/s]

,depth,learning_rate,nDCDG_cv,n_estimators,reg_lambda
26,1.0,0.010,0.851568,10.0,0.0010
25,1.0,0.010,0.851568,10.0,0.0001
0,1.0,0.001,0.851530,10.0,0.0001
7,1.0,0.001,0.851530,25.0,0.0100
13,1.0,0.001,0.851530,50.0,0.1000
8,1.0,0.001,0.851530,25.0,0.1000
1,1.0,0.001,0.851530,10.0,0.0010
6,1.0,0.001,0.851530,25.0,0.0010
5,1.0,0.001,0.851530,25.0,0.0001
3,1.0,0.001,0.851530,10.0,0.1000


## Сравнение результатов

Оказалось что для данной задачи больше всего подходит модель XGBRanker c функцией потерь `nDCG`.

Написал свой гридсерч `grid_search` для подбора параметров на кросс-валидации потому что так было удобнее и т.к все учится довольно быстро

### xgboost
- Топ 1 – XGBRanker c `nDCG`, как кажется и должно быть учитываю что наша метрики – nDCG :)
- Топ 2 – XGBRegressor с обычным `MSE`
- То есть `pointwise` себя тут неплохо показал
- По времени обучения – XGBRegressor кажется самый долгий. Но разница не особо заметная. Подбор параметров для всех xgb моделей происходил за [14, 19] мин

### catboost
- Показал себя хуже чем xgboost в данном случае. Но я думаю что это связано с тем что модели все-таки с разными гипер-параметрами мерялись
- Наверное действительно стоило добавить кат.фичи
- Pairwise в данном случае себя показал хуже pointwise
